<a href="https://colab.research.google.com/github/bhavani12860/Fmml-labs/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [2]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [3]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [5]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [6]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

NameError: name 'alltraindata' is not defined

What is the accuracy of our classifiers on the train dataset?

In [7]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

NameError: name 'traindata' is not defined

For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [9]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

NameError: name 'alltraindata' is not defined

### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

A good percentage to reserve for the validation set depends on the dataset size and complexity, but here are some general guidelines:

- For small datasets (<1000 samples), use 20-30% for validation.
- For medium datasets (1000-10000 samples), use 15-25% for validation.
- For large datasets (>10000 samples), use 10-20% for validation.

Reserving too little data for validation (e.g., <10%) may lead to high variability in accuracy estimates, while reserving too much (e.g., >30%) may reduce the training set size, potentially harming model performance.

Consider the following:

- If your dataset is very small, you may want to use a larger validation set to ensure reliable accuracy estimates.
- If your dataset is large, you may be able to get away with a smaller validation set.
- If your model is complex or has many hyperparameters, you may want to use a larger validation set to ensure robustness.

Remember, these are general guidelines. Experiment with different train-validation splits to find the optimal balance for your specific problem.

The size of the train and validation sets significantly impacts the accuracy prediction on the test set using the validation set:

*Train Set Size:*

1. *Small train set*: Limited data leads to poor model generalization, making it challenging to predict test accuracy using the validation set.
2. *Large train set*: More data allows the model to learn better, improving the accuracy prediction on the test set using the validation set.

*Validation Set Size:*

1. *Small validation set*: Insufficient data leads to high variability in accuracy estimates, making it unreliable for predicting test accuracy.
2. *Large validation set*: More data provides a more accurate and stable estimate of test accuracy, but may lead to overfitting if too large.

*Ideal Scenario:*

1. *Sufficient train set size*: Large enough to train a robust model.
2. *Moderate validation set size*: Large enough to provide a reliable estimate, but small enough to avoid overfitting.

*Key Takeaways:*

1. A large train set and moderate validation set size provide the best accuracy prediction on the test set.
2. A small train set or validation set can lead to poor accuracy prediction.
3. Experiment with different train-validation splits to find the optimal balance for your specific problem.

If you increase the percentage of the validation set:

1. _More accurate estimate_: A larger validation set provides a more accurate estimate of the model's performance, as it's based on a larger sample of data.
2. _Reduced variability_: The estimated accuracy will have lower variability, making it a more reliable indicator of the model's performance.
3. _Potential overfitting_: However, if the validation set is too large, the model may start to overfit to the validation set, rather than generalizing well to unseen data.

If you reduce the percentage of the validation set:

1. _Less accurate estimate_: A smaller validation set provides a less accurate estimate of the model's performance, as it's based on a smaller sample of data.
2. _Increased variability_: The estimated accuracy will have higher variability, making it a less reliable indicator of the model's performance.
3. _Reduced overfitting_: On the other hand, a smaller validation set can help reduce overfitting, as the model has fewer opportunities to overfit to the validation set.

In general, a common practice is to use:

- 80% for training
- 10% for validation
- 10% for testing

However, the optimal split depends on the specific problem, dataset size, and model complexity. Experimenting with different splits can help find the best balance for your specific use case.

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


While increasing iterations can help improve the estimate of test accuracy, it has limitations when dealing with very small train or validation datasets.

Small train dataset:

1. _Overfitting_: Increasing iterations can exacerbate overfitting, as the model may become too specialized to the small training set.
2. _Limited generalization_: A small train dataset may not capture the full range of patterns and variability in the data, limiting the model's ability to generalize.

Small validation dataset:

1. _Unreliable estimate_: A small validation set may not provide a reliable estimate of test accuracy, even with increased iterations.
2. _High variance_: The estimated accuracy may have high variance, making it difficult to trust the results.

To deal with very small datasets, consider:

1. _Data augmentation_: Artificially increase the dataset size by generating new samples through transformations or simulations.
2. _Transfer learning_: Leverage pre-trained models and fine-tune them on your small dataset.
3. _Regularization techniques_: Use regularization methods (e.g., L1, L2, dropout) to prevent overfitting.
4. _Collect more data_: If possible, collect more data to increase the train and validation set sizes.
5. _Use simpler models_: Consider using simpler models that are less prone to overfitting.

Increasing iterations can help, but it's essential to address the underlying challenges of small datasets through these strategies.

The number of iterations in cross-validation can impact the estimate of test accuracy:

1. *Increased iterations*: Generally, increasing the number of iterations (e.g., k in k-fold cross-validation) provides a more accurate and stable estimate of test accuracy.
2. *Reduced variability*: More iterations reduce the variability in the estimated accuracy, providing a more reliable estimate.
3. *Convergence*: As the number of iterations increases, the estimate converges to the true test accuracy.

However, there are diminishing returns:

1. *Computational cost*: Increasing iterations comes at a computational cost, which may be significant for large datasets.
2. *Overfitting*: Excessive iterations can lead to overfitting, especially if the model is complex.

Optimal number of iterations:

1. *Typical range*: 5-10 iterations (e.g., 5-fold or 10-fold cross-validation) often provide a good balance between accuracy and computational cost.
2. *Depends on data*: The optimal number of iterations depends on the dataset size, complexity, and noise level.
3. *Experimentation*: Experiment with different numbers of iterations to find the sweet spot for your specific problem.

In summary, increasing the number of iterations generally improves the estimate, but there are diminishing returns due to computational cost and potential overfitting. Experiment to find the optimal number of iterations for your specific problem.

Yes, averaging the validation accuracy across multiple splits can provide a more accurate estimate of the test accuracy. This is because:

1. Reduced variability: Cross-validation reduces the variability in the estimated accuracy, providing a more stable and reliable estimate.
2. Better generalization: By training and evaluating on multiple splits, you can identify models that generalize well to unseen data, which is a better indicator of test accuracy.
3. Unbiased estimate: Cross-validation provides an unbiased estimate of the test accuracy, as it is based on multiple independent evaluations.

However, keep in mind that:

1. Cross-validation is not a guarantee: While cross-validation can provide a more accurate estimate, it's not a guarantee of test accuracy.
2. Dependence on data: The quality of the estimate depends on the quality and representativeness of the data.
3. Computational cost: Cross-validation can be computationally expensive, especially for large datasets.

To further improve the estimate, consider:

1. Using a large number of splits (e.g., k=10 or more)
2. Using stratified cross-validation for imbalanced datasets
3. Using techniques like bootstrapping or bagging to further reduce variability
4. Evaluating on a separate test set, if possible

By using cross-validation and considering these factors, you can obtain a more accurate estimate of your model's test accuracy.

Yes, averaging the validation accuracy across multiple splits can provide more consistent results. This approach is known as cross-validation. By training and evaluating the model on multiple splits, you can:

1. Reduce overfitting: Models may perform well on a single split due to chance or overfitting. Cross-validation helps identify models that generalize well across multiple splits.
2. Increase robustness: Averaging accuracy across splits provides a more robust estimate of the model's performance, reducing the impact of individual split variations.
3. Improve reliability: Cross-validation helps ensure that the results are not due to a single lucky or unlucky split.

Common cross-validation techniques include:

1. K-fold cross-validation: Divide the data into k subsets, train on k-1 subsets, and evaluate on the remaining subset. Repeat for each subset.
2. Stratified cross-validation: Similar to k-fold, but maintains the class balance in each subset.
3. Leave-one-out cross-validation: Train on all data points except one, evaluate on that point, and repeat for all points.

By averaging the validation accuracy across multiple splits, you can obtain a more reliable and robust estimate of your model's performance.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.